In [2]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m,x): # added x input to forestall error messages
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m,x): # added x input to forestall error messages
    exn = exNo3c(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.

def normalJ(n,m,x): # added x input to forestall error messages
    f = J(n,m,x)
    return f.truncate(n+1)

def Jcoefficient(n,m,x):
    f = J(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m,x): # added x input to forestall error messages
    def xjNewStep(n,m):
        f=J(n,m,x)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m,x):return L(xjNew(n,m,x)/x) # added x input to forestall error messages

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def laurent_to_poly(f):
    fv=f.valuation()
    sf=f.shift(-fv)
    return sf

def laurent_constant(f): # for f  a Laurent series
    fv=f.valuation()
    pf=laurent_to_poly(f)
    return polynomialCoefficient(-fv,pf)

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [ ]:
# made this in 'redo j constants 28dec22 nu2.ipynb' with earlier code
# output snipped
polydata=[]
numericalfactordata=[]
broadhurst_values=[]
tests=[]
import time
for k in [1..100]:
    start=time.time()/60
    data=[]
    for m in [3..3*k+3]:
        cst=laurent_constant(j(k,m)^k) #earlier code (no 'x' input) but see first cell
        data=data+[[m,cst]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[k,rl]]
    nf=ZZ(numericalfactor(rl))
    test=ZZ(nf/24-broadhurst(k))
    tests=tests+[test]
    finish=time.time()/60
    print("(k,test,minutes): ",(k,test,finish-start))
print("===============================================================================")
print("tests:")
print(tests)
import pickle
wfile = open('/Users/barrybrent/28dec22no5.txt','wb')
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()
print("tests:")
print(tests)

In [3]:
import pickle
rfile = open('/Users/barrybrent/28dec22no5.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

100


In [4]:
import pickle
rfile = open('/Users/barrybrent/28dec22no5.txt','rb')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for n in [3..100]:
    p=prime(n)
    if (p>len(s))==False:
        power=s[p-1][0] # = p.
        poly=D(s[p-1][1]) # poly(m) = cst term of j_m^power
        for a in [1..200]:
            m=p^a
            polym=evaluate(poly,m)
            o=order(p,polym)
            test=o-(a*p-2)
            tupl=(p,a,power,test)
            data=data+[test]
            print(tupl)
print(data)

(5, 1, 5, 0)
(5, 2, 5, 0)
(5, 3, 5, 0)
(5, 4, 5, 0)
(5, 5, 5, 0)
(5, 6, 5, 0)
(5, 7, 5, 0)
(5, 8, 5, 0)
(5, 9, 5, 0)
(5, 10, 5, 0)
(5, 11, 5, 0)
(5, 12, 5, 0)
(5, 13, 5, 0)
(5, 14, 5, 0)
(5, 15, 5, 0)
(5, 16, 5, 0)
(5, 17, 5, 0)
(5, 18, 5, 0)
(5, 19, 5, 0)
(5, 20, 5, 0)
(5, 21, 5, 0)
(5, 22, 5, 0)
(5, 23, 5, 0)
(5, 24, 5, 0)
(5, 25, 5, 0)
(5, 26, 5, 0)
(5, 27, 5, 0)
(5, 28, 5, 0)
(5, 29, 5, 0)
(5, 30, 5, 0)
(5, 31, 5, 0)
(5, 32, 5, 0)
(5, 33, 5, 0)
(5, 34, 5, 0)
(5, 35, 5, 0)
(5, 36, 5, 0)
(5, 37, 5, 0)
(5, 38, 5, 0)
(5, 39, 5, 0)
(5, 40, 5, 0)
(5, 41, 5, 0)
(5, 42, 5, 0)
(5, 43, 5, 0)
(5, 44, 5, 0)
(5, 45, 5, 0)
(5, 46, 5, 0)
(5, 47, 5, 0)
(5, 48, 5, 0)
(5, 49, 5, 0)
(5, 50, 5, 0)
(5, 51, 5, 0)
(5, 52, 5, 0)
(5, 53, 5, 0)
(5, 54, 5, 0)
(5, 55, 5, 0)
(5, 56, 5, 0)
(5, 57, 5, 0)
(5, 58, 5, 0)
(5, 59, 5, 0)
(5, 60, 5, 0)
(5, 61, 5, 0)
(5, 62, 5, 0)
(5, 63, 5, 0)
(5, 64, 5, 0)
(5, 65, 5, 0)
(5, 66, 5, 0)
(5, 67, 5, 0)
(5, 68, 5, 0)
(5, 69, 5, 0)
(5, 70, 5, 0)
(5, 71, 5, 0)
(5, 72, 5, 0)
(

(11, 200, 11, 0)
(13, 1, 13, 0)
(13, 2, 13, 0)
(13, 3, 13, 0)
(13, 4, 13, 0)
(13, 5, 13, 0)
(13, 6, 13, 0)
(13, 7, 13, 0)
(13, 8, 13, 0)
(13, 9, 13, 0)
(13, 10, 13, 0)
(13, 11, 13, 0)
(13, 12, 13, 0)
(13, 13, 13, 0)
(13, 14, 13, 0)
(13, 15, 13, 0)
(13, 16, 13, 0)
(13, 17, 13, 0)
(13, 18, 13, 0)
(13, 19, 13, 0)
(13, 20, 13, 0)
(13, 21, 13, 0)
(13, 22, 13, 0)
(13, 23, 13, 0)
(13, 24, 13, 0)
(13, 25, 13, 0)
(13, 26, 13, 0)
(13, 27, 13, 0)
(13, 28, 13, 0)
(13, 29, 13, 0)
(13, 30, 13, 0)
(13, 31, 13, 0)
(13, 32, 13, 0)
(13, 33, 13, 0)
(13, 34, 13, 0)
(13, 35, 13, 0)
(13, 36, 13, 0)
(13, 37, 13, 0)
(13, 38, 13, 0)
(13, 39, 13, 0)
(13, 40, 13, 0)
(13, 41, 13, 0)
(13, 42, 13, 0)
(13, 43, 13, 0)
(13, 44, 13, 0)
(13, 45, 13, 0)
(13, 46, 13, 0)
(13, 47, 13, 0)
(13, 48, 13, 0)
(13, 49, 13, 0)
(13, 50, 13, 0)
(13, 51, 13, 0)
(13, 52, 13, 0)
(13, 53, 13, 0)
(13, 54, 13, 0)
(13, 55, 13, 0)
(13, 56, 13, 0)
(13, 57, 13, 0)
(13, 58, 13, 0)
(13, 59, 13, 0)
(13, 60, 13, 0)
(13, 61, 13, 0)
(13, 62, 13, 0)


(19, 112, 19, 0)
(19, 113, 19, 0)
(19, 114, 19, 0)
(19, 115, 19, 0)
(19, 116, 19, 0)
(19, 117, 19, 0)
(19, 118, 19, 0)
(19, 119, 19, 0)
(19, 120, 19, 0)
(19, 121, 19, 0)
(19, 122, 19, 0)
(19, 123, 19, 0)
(19, 124, 19, 0)
(19, 125, 19, 0)
(19, 126, 19, 0)
(19, 127, 19, 0)
(19, 128, 19, 0)
(19, 129, 19, 0)
(19, 130, 19, 0)
(19, 131, 19, 0)
(19, 132, 19, 0)
(19, 133, 19, 0)
(19, 134, 19, 0)
(19, 135, 19, 0)
(19, 136, 19, 0)
(19, 137, 19, 0)
(19, 138, 19, 0)
(19, 139, 19, 0)
(19, 140, 19, 0)
(19, 141, 19, 0)
(19, 142, 19, 0)
(19, 143, 19, 0)
(19, 144, 19, 0)
(19, 145, 19, 0)
(19, 146, 19, 0)
(19, 147, 19, 0)
(19, 148, 19, 0)
(19, 149, 19, 0)
(19, 150, 19, 0)
(19, 151, 19, 0)
(19, 152, 19, 0)
(19, 153, 19, 0)
(19, 154, 19, 0)
(19, 155, 19, 0)
(19, 156, 19, 0)
(19, 157, 19, 0)
(19, 158, 19, 0)
(19, 159, 19, 0)
(19, 160, 19, 0)
(19, 161, 19, 0)
(19, 162, 19, 0)
(19, 163, 19, 0)
(19, 164, 19, 0)
(19, 165, 19, 0)
(19, 166, 19, 0)
(19, 167, 19, 0)
(19, 168, 19, 0)
(19, 169, 19, 0)
(19, 170, 19, 

(31, 7, 31, 0)
(31, 8, 31, 0)
(31, 9, 31, 0)
(31, 10, 31, 0)
(31, 11, 31, 0)
(31, 12, 31, 0)
(31, 13, 31, 0)
(31, 14, 31, 0)
(31, 15, 31, 0)
(31, 16, 31, 0)
(31, 17, 31, 0)
(31, 18, 31, 0)
(31, 19, 31, 0)
(31, 20, 31, 0)
(31, 21, 31, 0)
(31, 22, 31, 0)
(31, 23, 31, 0)
(31, 24, 31, 0)
(31, 25, 31, 0)
(31, 26, 31, 0)
(31, 27, 31, 0)
(31, 28, 31, 0)
(31, 29, 31, 0)
(31, 30, 31, 0)
(31, 31, 31, 0)
(31, 32, 31, 0)
(31, 33, 31, 0)
(31, 34, 31, 0)
(31, 35, 31, 0)
(31, 36, 31, 0)
(31, 37, 31, 0)
(31, 38, 31, 0)
(31, 39, 31, 0)
(31, 40, 31, 0)
(31, 41, 31, 0)
(31, 42, 31, 0)
(31, 43, 31, 0)
(31, 44, 31, 0)
(31, 45, 31, 0)
(31, 46, 31, 0)
(31, 47, 31, 0)
(31, 48, 31, 0)
(31, 49, 31, 0)
(31, 50, 31, 0)
(31, 51, 31, 0)
(31, 52, 31, 0)
(31, 53, 31, 0)
(31, 54, 31, 0)
(31, 55, 31, 0)
(31, 56, 31, 0)
(31, 57, 31, 0)
(31, 58, 31, 0)
(31, 59, 31, 0)
(31, 60, 31, 0)
(31, 61, 31, 0)
(31, 62, 31, 0)
(31, 63, 31, 0)
(31, 64, 31, 0)
(31, 65, 31, 0)
(31, 66, 31, 0)
(31, 67, 31, 0)
(31, 68, 31, 0)
(31, 69, 31

(41, 120, 41, 0)
(41, 121, 41, 0)
(41, 122, 41, 0)
(41, 123, 41, 0)
(41, 124, 41, 0)
(41, 125, 41, 0)
(41, 126, 41, 0)
(41, 127, 41, 0)
(41, 128, 41, 0)
(41, 129, 41, 0)
(41, 130, 41, 0)
(41, 131, 41, 0)
(41, 132, 41, 0)
(41, 133, 41, 0)
(41, 134, 41, 0)
(41, 135, 41, 0)
(41, 136, 41, 0)
(41, 137, 41, 0)
(41, 138, 41, 0)
(41, 139, 41, 0)
(41, 140, 41, 0)
(41, 141, 41, 0)
(41, 142, 41, 0)
(41, 143, 41, 0)
(41, 144, 41, 0)
(41, 145, 41, 0)
(41, 146, 41, 0)
(41, 147, 41, 0)
(41, 148, 41, 0)
(41, 149, 41, 0)
(41, 150, 41, 0)
(41, 151, 41, 0)
(41, 152, 41, 0)
(41, 153, 41, 0)
(41, 154, 41, 0)
(41, 155, 41, 0)
(41, 156, 41, 0)
(41, 157, 41, 0)
(41, 158, 41, 0)
(41, 159, 41, 0)
(41, 160, 41, 0)
(41, 161, 41, 0)
(41, 162, 41, 0)
(41, 163, 41, 0)
(41, 164, 41, 0)
(41, 165, 41, 0)
(41, 166, 41, 0)
(41, 167, 41, 0)
(41, 168, 41, 0)
(41, 169, 41, 0)
(41, 170, 41, 0)
(41, 171, 41, 0)
(41, 172, 41, 0)
(41, 173, 41, 0)
(41, 174, 41, 0)
(41, 175, 41, 0)
(41, 176, 41, 0)
(41, 177, 41, 0)
(41, 178, 41, 

(53, 20, 53, 0)
(53, 21, 53, 0)
(53, 22, 53, 0)
(53, 23, 53, 0)
(53, 24, 53, 0)
(53, 25, 53, 0)
(53, 26, 53, 0)
(53, 27, 53, 0)
(53, 28, 53, 0)
(53, 29, 53, 0)
(53, 30, 53, 0)
(53, 31, 53, 0)
(53, 32, 53, 0)
(53, 33, 53, 0)
(53, 34, 53, 0)
(53, 35, 53, 0)
(53, 36, 53, 0)
(53, 37, 53, 0)
(53, 38, 53, 0)
(53, 39, 53, 0)
(53, 40, 53, 0)
(53, 41, 53, 0)
(53, 42, 53, 0)
(53, 43, 53, 0)
(53, 44, 53, 0)
(53, 45, 53, 0)
(53, 46, 53, 0)
(53, 47, 53, 0)
(53, 48, 53, 0)
(53, 49, 53, 0)
(53, 50, 53, 0)
(53, 51, 53, 0)
(53, 52, 53, 0)
(53, 53, 53, 0)
(53, 54, 53, 0)
(53, 55, 53, 0)
(53, 56, 53, 0)
(53, 57, 53, 0)
(53, 58, 53, 0)
(53, 59, 53, 0)
(53, 60, 53, 0)
(53, 61, 53, 0)
(53, 62, 53, 0)
(53, 63, 53, 0)
(53, 64, 53, 0)
(53, 65, 53, 0)
(53, 66, 53, 0)
(53, 67, 53, 0)
(53, 68, 53, 0)
(53, 69, 53, 0)
(53, 70, 53, 0)
(53, 71, 53, 0)
(53, 72, 53, 0)
(53, 73, 53, 0)
(53, 74, 53, 0)
(53, 75, 53, 0)
(53, 76, 53, 0)
(53, 77, 53, 0)
(53, 78, 53, 0)
(53, 79, 53, 0)
(53, 80, 53, 0)
(53, 81, 53, 0)
(53, 82,

(61, 121, 61, 0)
(61, 122, 61, 0)
(61, 123, 61, 0)
(61, 124, 61, 0)
(61, 125, 61, 0)
(61, 126, 61, 0)
(61, 127, 61, 0)
(61, 128, 61, 0)
(61, 129, 61, 0)
(61, 130, 61, 0)
(61, 131, 61, 0)
(61, 132, 61, 0)
(61, 133, 61, 0)
(61, 134, 61, 0)
(61, 135, 61, 0)
(61, 136, 61, 0)
(61, 137, 61, 0)
(61, 138, 61, 0)
(61, 139, 61, 0)
(61, 140, 61, 0)
(61, 141, 61, 0)
(61, 142, 61, 0)
(61, 143, 61, 0)
(61, 144, 61, 0)
(61, 145, 61, 0)
(61, 146, 61, 0)
(61, 147, 61, 0)
(61, 148, 61, 0)
(61, 149, 61, 0)
(61, 150, 61, 0)
(61, 151, 61, 0)
(61, 152, 61, 0)
(61, 153, 61, 0)
(61, 154, 61, 0)
(61, 155, 61, 0)
(61, 156, 61, 0)
(61, 157, 61, 0)
(61, 158, 61, 0)
(61, 159, 61, 0)
(61, 160, 61, 0)
(61, 161, 61, 0)
(61, 162, 61, 0)
(61, 163, 61, 0)
(61, 164, 61, 0)
(61, 165, 61, 0)
(61, 166, 61, 0)
(61, 167, 61, 0)
(61, 168, 61, 0)
(61, 169, 61, 0)
(61, 170, 61, 0)
(61, 171, 61, 0)
(61, 172, 61, 0)
(61, 173, 61, 0)
(61, 174, 61, 0)
(61, 175, 61, 0)
(61, 176, 61, 0)
(61, 177, 61, 0)
(61, 178, 61, 0)
(61, 179, 61, 

(73, 18, 73, 0)
(73, 19, 73, 0)
(73, 20, 73, 0)
(73, 21, 73, 0)
(73, 22, 73, 0)
(73, 23, 73, 0)
(73, 24, 73, 0)
(73, 25, 73, 0)
(73, 26, 73, 0)
(73, 27, 73, 0)
(73, 28, 73, 0)
(73, 29, 73, 0)
(73, 30, 73, 0)
(73, 31, 73, 0)
(73, 32, 73, 0)
(73, 33, 73, 0)
(73, 34, 73, 0)
(73, 35, 73, 0)
(73, 36, 73, 0)
(73, 37, 73, 0)
(73, 38, 73, 0)
(73, 39, 73, 0)
(73, 40, 73, 0)
(73, 41, 73, 0)
(73, 42, 73, 0)
(73, 43, 73, 0)
(73, 44, 73, 0)
(73, 45, 73, 0)
(73, 46, 73, 0)
(73, 47, 73, 0)
(73, 48, 73, 0)
(73, 49, 73, 0)
(73, 50, 73, 0)
(73, 51, 73, 0)
(73, 52, 73, 0)
(73, 53, 73, 0)
(73, 54, 73, 0)
(73, 55, 73, 0)
(73, 56, 73, 0)
(73, 57, 73, 0)
(73, 58, 73, 0)
(73, 59, 73, 0)
(73, 60, 73, 0)
(73, 61, 73, 0)
(73, 62, 73, 0)
(73, 63, 73, 0)
(73, 64, 73, 0)
(73, 65, 73, 0)
(73, 66, 73, 0)
(73, 67, 73, 0)
(73, 68, 73, 0)
(73, 69, 73, 0)
(73, 70, 73, 0)
(73, 71, 73, 0)
(73, 72, 73, 0)
(73, 73, 73, 0)
(73, 74, 73, 0)
(73, 75, 73, 0)
(73, 76, 73, 0)
(73, 77, 73, 0)
(73, 78, 73, 0)
(73, 79, 73, 0)
(73, 80,

(83, 122, 83, 0)
(83, 123, 83, 0)
(83, 124, 83, 0)
(83, 125, 83, 0)
(83, 126, 83, 0)
(83, 127, 83, 0)
(83, 128, 83, 0)
(83, 129, 83, 0)
(83, 130, 83, 0)
(83, 131, 83, 0)
(83, 132, 83, 0)
(83, 133, 83, 0)
(83, 134, 83, 0)
(83, 135, 83, 0)
(83, 136, 83, 0)
(83, 137, 83, 0)
(83, 138, 83, 0)
(83, 139, 83, 0)
(83, 140, 83, 0)
(83, 141, 83, 0)
(83, 142, 83, 0)
(83, 143, 83, 0)
(83, 144, 83, 0)
(83, 145, 83, 0)
(83, 146, 83, 0)
(83, 147, 83, 0)
(83, 148, 83, 0)
(83, 149, 83, 0)
(83, 150, 83, 0)
(83, 151, 83, 0)
(83, 152, 83, 0)
(83, 153, 83, 0)
(83, 154, 83, 0)
(83, 155, 83, 0)
(83, 156, 83, 0)
(83, 157, 83, 0)
(83, 158, 83, 0)
(83, 159, 83, 0)
(83, 160, 83, 0)
(83, 161, 83, 0)
(83, 162, 83, 0)
(83, 163, 83, 0)
(83, 164, 83, 0)
(83, 165, 83, 0)
(83, 166, 83, 0)
(83, 167, 83, 0)
(83, 168, 83, 0)
(83, 169, 83, 0)
(83, 170, 83, 0)
(83, 171, 83, 0)
(83, 172, 83, 0)
(83, 173, 83, 0)
(83, 174, 83, 0)
(83, 175, 83, 0)
(83, 176, 83, 0)
(83, 177, 83, 0)
(83, 178, 83, 0)
(83, 179, 83, 0)
(83, 180, 83, 